# Hearing a Rectangle (Momenteschätzer)

In [1]:
# Imports
#
# C:\Users\sulta\AppData\Local\Programs\Python\Python310\Scripts\pip install scipy
#
import numpy as np
from scipy import stats, optimize
from scipy.optimize import minimize
from scipy.optimize import curve_fit

In [2]:
# Parameters of the "true" rectangle
a_true = 1.0
b_true = 3.0
pi = np.pi

# sollte locker für 1000 Eigenwerte reichen
m_max, n_max = 800, 800

ms = np.arange(1, m_max + 1)
ns = np.arange(1, n_max + 1)
M, N = np.meshgrid(ms, ns, indexing="ij")

# Dirichlet-Eigenwerte: λ_{m,n} = π²(m²/a² + n²/b²)
lambdas = pi**2 * (M**2 / a_true**2 + N**2 / b_true**2)

# Flatten & sort
l_flat = lambdas.ravel()
sort_idx = np.argsort(l_flat)
l_sorted = l_flat[sort_idx]

Nvals = 10000
eigvals = l_sorted[:Nvals]

print("Erste 12 Eigenwerte (gerundet):")
print([round(float(x), 4) for x in eigvals[:12]])

Erste 12 Eigenwerte (gerundet):
[10.9662, 14.2561, 19.7392, 27.4156, 37.2852, 40.575, 43.8649, 49.348, 49.348, 57.0244, 63.6041, 66.894]


In [3]:
# ---------------------------------------------------------
# Step 1: Fläche aus Weyl via linearer Regression
# N(λ) ~ (ab)/(4π) * λ => k ≈ α λ + β
# ---------------------------------------------------------
k = np.arange(1, Nvals + 1)

start = 300  # nur Hochfrequenz für Fit
lam_seg = eigvals[start-1:]
k_seg = k[start-1:]

slope, intercept, r_value, p_value, std_err = stats.linregress(lam_seg, k_seg)
ab_est = 4 * pi * slope

print(f"\nLinReg-Slope α ≈ {slope:.4f}")
print(f"geschätzte Fläche ab_est ≈ {ab_est:.4f} (true ab = {a_true * b_true})")

# ---------------------------------------------------------
# Step 2: Aspektverhältnis R über Momente von m² und n²
# ---------------------------------------------------------

# (m,n) zu sortierten Eigenwerten rekonstruieren
m_idx, n_idx = np.unravel_index(sort_idx[:Nvals], M.shape)
m_sel = M[m_idx, n_idx]
n_sel = N[m_idx, n_idx]

m2_mean = np.mean(m_sel**2)
n2_mean = np.mean(n_sel**2)

R_hat = np.sqrt(m2_mean / n2_mean)

print(f"\ngeschätztes Aspektverhältnis R_hat ≈ {R_hat:.4f} "
      f"(true R = {a_true / b_true:.4f})")

# ---------------------------------------------------------
# Step 3: a und b aus ab_est und R_hat
# ---------------------------------------------------------
a_hat = np.sqrt(ab_est) * np.sqrt(R_hat)
b_hat = np.sqrt(ab_est) / np.sqrt(R_hat)

print("\nRekonstruierte Seitenlängen:")
print(f"a_hat ≈ {a_hat:.4f} (true a = {a_true})")
print(f"b_hat ≈ {b_hat:.4f} (true b = {b_true})")



LinReg-Slope α ≈ 0.2364
geschätzte Fläche ab_est ≈ 2.9703 (true ab = 3.0)

geschätztes Aspektverhältnis R_hat ≈ 0.3348 (true R = 0.3333)

Rekonstruierte Seitenlängen:
a_hat ≈ 0.9973 (true a = 1.0)
b_hat ≈ 2.9785 (true b = 3.0)
